In [1]:
import pandas as pd
import numpy as np

In [2]:
raw = pd.read_excel("new_table.xlsx", header = 0, sheet_name="new_table")
raw.head()

,식당(ID),식당명,지점명,지역명,어워드정보설명,(RTI)지수,온라인화진행여부,수용태세지수,인기도,트립어드바이저평점,씨트립평점,네이버평점
0,130027,이브다방,NaN,종로구,NaN,0.910544,N,0.246,NaN,NaN,NaN,5.0
1,134565,금문장,NaN,종로구,NaN,4.351588,N,0.218,0.33,NaN,NaN,5.0
2,134973,원조동대문곱창,NaN,종로구,NaN,4.356354,Y,0.235,0.20,NaN,NaN,5.0
3,135069,도스타코스,NaN,종로구,NaN,3.411680,N,0.217,NaN,NaN,NaN,5.0
4,135516,순대와 감자탕,NaN,종로구,NaN,4.071734,N,0.217,0.02,NaN,NaN,5.0


In [3]:
my_area = raw[raw['지역명'].isin(['종로구','양천구','용산구','서대문구','성북구'])][['식당(ID)','식당명', '지점명','지역명','네이버평점']]

my_area.head(10)

,식당(ID),식당명,지점명,지역명,네이버평점
0,130027,이브다방,NaN,종로구,5.0
1,134565,금문장,NaN,종로구,5.0
2,134973,원조동대문곱창,NaN,종로구,5.0
3,135069,도스타코스,NaN,종로구,5.0
4,135516,순대와 감자탕,NaN,종로구,5.0
5,136142,원조낭창집,NaN,종로구,5.0
6,137658,커피 디저트 라이스,NaN,종로구,5.0
7,138084,피플호프커피,NaN,종로구,5.0
8,138917,이화식당,NaN,종로구,5.0
9,139444,더 소호,NaN,종로구,5.0


In [4]:
import numpy as np

# NaN 값을 처리하기 위해 numpy를 import합니다.

my_area['식당명+지점명'] = np.where(
    my_area['지점명'].notna(),
    my_area['식당명'].astype(str) + " " + my_area['지점명'].astype(str),
    my_area['지역명'] + " " + my_area['식당명'].astype(str)
)

my_area.head(10)


,식당(ID),식당명,지점명,지역명,네이버평점,식당명+지점명
0,130027,이브다방,NaN,종로구,5.0,종로구 이브다방
1,134565,금문장,NaN,종로구,5.0,종로구 금문장
2,134973,원조동대문곱창,NaN,종로구,5.0,종로구 원조동대문곱창
3,135069,도스타코스,NaN,종로구,5.0,종로구 도스타코스
4,135516,순대와 감자탕,NaN,종로구,5.0,종로구 순대와 감자탕
5,136142,원조낭창집,NaN,종로구,5.0,종로구 원조낭창집
6,137658,커피 디저트 라이스,NaN,종로구,5.0,종로구 커피 디저트 라이스
7,138084,피플호프커피,NaN,종로구,5.0,종로구 피플호프커피
8,138917,이화식당,NaN,종로구,5.0,종로구 이화식당
9,139444,더 소호,NaN,종로구,5.0,종로구 더 소호


In [5]:
name_list = my_area['식당명+지점명'].tolist()
name_list = name_list
name_list[:10]

['종로구 이브다방',
 '종로구 금문장',
 '종로구 원조동대문곱창',
 '종로구 도스타코스',
 '종로구 순대와 감자탕',
 '종로구 원조낭창집',
 '종로구 커피 디저트 라이스',
 '종로구 피플호프커피',
 '종로구 이화식당',
 '종로구 더 소호']

In [6]:
import requests

res_info_list = []
no_place = []

for searching in name_list:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {
        "Authorization": "KakaoAK b9f489946ca62947a1d9b5320150e4a7"
    }
    places = requests.get(url, headers=headers).json()['documents']
    
    if len(places) <= 0:
        places = {'address_name': np.nan,
                 'category_group_code': np.nan,
                 'category_group_name': np.nan,
                 'category_name': np.nan,
                 'distance': np.nan,
                 'id': np.nan,
                 'phone': np.nan,
                 'place_name': np.nan,
                 'place_url': np.nan,
                 'road_address_name': np.nan,
                 'x': np.nan,
                 'y': np.nan}
        res_info_list.append(places)
        
    else:
        res_info_list.append(places[0])

In [7]:
len(res_info_list)

26146

In [19]:
res_info_df = pd.DataFrame(res_info_list)
res_info_df['평점'] = raw['네이버평점']
res_info_df.loc[res_info_df['address_name'].isna(), '평점'] = np.nan

In [20]:
res_info_df[:10]

,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,평점
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,서울 종로구 숭인동 61-30,FD6,음식점,음식점 > 중식 > 중국요리,,24058091,,금문장,http://place.map.kakao.com/24058091,서울 종로구 지봉로8길 45,127.016174788877,37.5766296538935,5.0
2,서울 은평구 녹번동 131-160,FD6,음식점,"음식점 > 한식 > 육류,고기 > 곱창,막창",,17067510,02-351-3367,원조동대문곱창,http://place.map.kakao.com/17067510,서울 은평구 진흥로 150,126.92664770684712,37.60864723978791,5.0
3,서울 중구 서소문동 47-2,FD6,음식점,"음식점 > 양식 > 멕시칸,브라질 > 도스타코스",,811720411,02-754-0201,도스타코스 시청배달점,http://place.map.kakao.com/811720411,서울 중구 서소문로 115,126.97368667695514,37.56335526620524,5.0
4,서울 종로구 청진동 3-3,FD6,음식점,음식점 > 한식 > 순대,,24725093,02-735-8055,장터 순대국파보쌈감자탕,http://place.map.kakao.com/24725093,서울 종로구 종로5길 40,126.98041551776817,37.572102539906425,5.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,서울 종로구 충신동 53-15,FD6,음식점,음식점 > 한식,,275835241,02-763-5433,이화식당,http://place.map.kakao.com/275835241,서울 종로구 율곡로19길 1,127.005130414835,37.5758377753911,5.0
9,서울 종로구 필운동 90-1,,,여행 > 숙박,,7836228,02-722-1999,더소호 레지던스,http://place.map.kakao.com/7836228,서울 종로구 필운대로 17,126.969271006812,37.5778881545215,5.0


### 2차 데이터 전처리

In [21]:
# 내 지역구가 아닌 지역구
not_my_area = ['강남구', '도봉구', '중구', '중랑구', '광진구', '마포구',
               '금천구', '서초구', '성동구', '동대문구', '송파구', '동작구', 
               '강서구', '노원구', '강동구', '영등포구', '강북구', '관악구', '은평구', '구로구']

# 내 지역구가 아닌 행의 인덱스를 찾음
rows_to_drop = res_info_df[res_info_df['address_name'].str.contains('|'.join(not_my_area), na=False)].index

# 해당 행을 삭제
res_info_df = res_info_df.drop(rows_to_drop)

# reset_index() 함수를 호출하여 인덱스를 다시 정렬
res_info_df = res_info_df.reset_index(drop=True)

In [22]:
# 카카오API로 겁색하지 못한 음식점 개수
res_info_df.isna().sum()

address_name            6255
category_group_code     6255
category_group_name     6255
category_name           6255
distance                6255
id                      6255
phone                   6255
place_name              6255
place_url               6255
road_address_name       6255
x                       6255
y                       6255
평점                     12089
dtype: int64

In [23]:
# 서울 아닌 행, NaN값 삭제
rows_to_drop2 = res_info_df[~res_info_df['address_name'].str.startswith('서울', na=False)].index
res_info_df = res_info_df.drop(rows_to_drop2)
res_info_df = res_info_df.reset_index(drop=True)

In [24]:
res_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18461 entries, 0 to 18460
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   address_name         18461 non-null  object 
 1   category_group_code  18461 non-null  object 
 2   category_group_name  18461 non-null  object 
 3   category_name        18461 non-null  object 
 4   distance             18461 non-null  object 
 5   id                   18461 non-null  object 
 6   phone                18461 non-null  object 
 7   place_name           18461 non-null  object 
 8   place_url            18461 non-null  object 
 9   road_address_name    18461 non-null  object 
 10  x                    18461 non-null  object 
 11  y                    18461 non-null  object 
 12  평점                   12677 non-null  float64
dtypes: float64(1), object(12)
memory usage: 1.8+ MB


In [25]:
# 음식점 아닌 행 삭제
rows_to_drop3 = res_info_df[~res_info_df['category_name'].str.startswith('음식점', na=False)].index
res_info_df = res_info_df.drop(rows_to_drop3)
res_info_df = res_info_df.reset_index(drop=True)

In [26]:
res_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17100 entries, 0 to 17099
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   address_name         17100 non-null  object 
 1   category_group_code  17100 non-null  object 
 2   category_group_name  17100 non-null  object 
 3   category_name        17100 non-null  object 
 4   distance             17100 non-null  object 
 5   id                   17100 non-null  object 
 6   phone                17100 non-null  object 
 7   place_name           17100 non-null  object 
 8   place_url            17100 non-null  object 
 9   road_address_name    17100 non-null  object 
 10  x                    17100 non-null  object 
 11  y                    17100 non-null  object 
 12  평점                   12355 non-null  float64
dtypes: float64(1), object(12)
memory usage: 1.7+ MB


### 공통컬럼만

In [27]:
df_final = res_info_df.iloc[:,[0,3,7,11,10,8,12]]
df_final.columns = ['도로명주소', '카테고리', '상호명', '위도(y)', '경도(x)', '사이트', '별점']
df_final

,도로명주소,카테고리,상호명,위도(y),경도(x),사이트,별점
0,서울 종로구 숭인동 61-30,음식점 > 중식 > 중국요리,금문장,37.5766296538935,127.016174788877,http://place.map.kakao.com/24058091,5.0
1,서울 종로구 청진동 3-3,음식점 > 한식 > 순대,장터 순대국파보쌈감자탕,37.572102539906425,126.98041551776817,http://place.map.kakao.com/24725093,5.0
2,서울 종로구 충신동 53-15,음식점 > 한식,이화식당,37.5758377753911,127.005130414835,http://place.map.kakao.com/275835241,5.0
3,서울 종로구 창신동 138-88,"음식점 > 한식 > 찌개,전골",오복양푼이탕,37.5734767156355,127.011550992964,http://place.map.kakao.com/773742635,5.0
4,서울 종로구 돈의동 153,"음식점 > 술집 > 호프,요리주점",하늘정원,37.57150914693,126.990772126865,http://place.map.kakao.com/282791167,5.0
...,...,...,...,...,...,...,...
17095,서울 성북구 종암동 65,"음식점 > 간식 > 제과,베이커리",오월의 밀밭,37.5980518705471,127.03258985498,http://place.map.kakao.com/2379438,NaN
17096,서울 종로구 재동 5-2,"음식점 > 간식 > 제과,베이커리",우드앤브릭 재동점,37.579545323669095,126.98421402219715,http://place.map.kakao.com/25752359,NaN
17097,서울 성북구 안암동5가 1-2,"음식점 > 간식 > 제과,베이커리",고대빵 본점,37.584331950617035,127.02752646650933,http://place.map.kakao.com/2086523497,NaN
17098,서울 성북구 정릉동 16-314,음식점 > 한식 > 국수 > 칼국수,정릉손칼국수,37.6022574023056,127.016751086703,http://place.map.kakao.com/16557826,NaN


In [28]:
df_final['카테고리'].value_counts()

음식점 > 카페                           1661
음식점 > 한식                           1614
음식점 > 술집 > 호프,요리주점                  882
음식점 > 한식 > 육류,고기                    872
음식점 > 분식                            633
                                   ... 
음식점 > 한식 > 국수 > 미사리밀빛초계국수             1
음식점 > 한식 > 육류,고기 > 족발,보쌈 > 놀부보쌈       1
음식점 > 퓨전요리 > 돼지게티                     1
음식점 > 패스트푸드 > KFC                     1
음식점 > 양식 > 피자 > 프레드피자                 1
Name: 카테고리, Length: 484, dtype: int64

### 최종 데이터프레임 내보내기

In [29]:
df_final.to_excel('식단품질정보_전처리완료_전체2.xlsx')